In [ ]:
import json
import glob
import os
from functools import reduce

import numpy as np
from matplotlib import pyplot as plt

from enum import IntEnum
from dataclasses import dataclass, field
from typing import List
from copy import deepcopy

from scobility import *

In [ ]:
itl = process_itl()

In [ ]:
itl.view_scobility()

In [ ]:
p = itl.players[274]
scores = 1 - np.array([(s.s_id in p.scores and p.scores[s.s_id].value or np.nan) for s in itl.ordering]) * Relationship.SCORE_SCALAR

p_bility = np.log2(itl.scobility)
p_scores = np.log2(0.001 + scores)

# data = np.vstack([itl.scobility, scores])
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].scatter(p_bility, p_scores)
ax[1].scatter(itl.scobility, scores)
#ax.set_ylim(0, 0.02)
plt.show()
    

In [ ]:
e_id = 321
p = itl.players[e_id]
scores = 1 - np.array([(s.s_id in p.scores and p.scores[s.s_id].value or np.nan) for s in itl.ordering]) * Relationship.SCORE_SCALAR

p_bility = np.log2(itl.scobility)
p_scores = np.log2(0.003 + scores)      # TODO: tune the offset from perfect

p_data = np.vstack([p_bility, p_scores])

# plotting is funny
z = np.sort(p_data, axis=-1)

# slope of line thru 0 as a starting point?
p_played = ~np.isnan(p_scores)
a = p_bility[p_played]
b = p_bility[p_played] - p_scores[p_played]
w = a # np.ones_like(a) # 1 - 1/(a + Relationship.WEIGHT_OFFSET)**2
m = np.vstack([np.ones_like(a), a])
coefs, resid = np.linalg.lstsq(m.T * w[:, np.newaxis], b * w, rcond=None)[:2]
b_eq = coefs @ m

# print([s.s_id for s in itl.ordering])
# print(p_bility)
# print(p_scores)
print(coefs)

fig, ax = plt.subplots(figsize=(6, 6))
plt.plot(a, b_eq, 'r')
plt.scatter(a, b)
#ax.set_ylim(0, 0.02)
plt.show()

p_quality = p_bility[p_played] - p_scores[p_played]
p_songs = np.array(itl.ordering)[p_played]
p_ranking = [z for z in zip([s for s in p_songs], [q for q in p_quality])]
p_ranking.sort(key=lambda x: x[1])

scobility_overall = sum(p_quality) / len(p_quality)
print(f'Funny Summary Number(tm) for {p.name} (#{p.e_id}): {scobility_overall:0.3f}')
if coefs[1] > 0:
    print(f'>>> You outperform your peers on harder charts. (m = {coefs[1]:0.3f})')
else:
    print(f'>>> You outperform your peers on easier charts. (m = {coefs[1]:0.3f})')

print(f'\nTop 5 Best Scores for {p.name} (#{p.e_id}):')
for s, v in reversed(p_ranking[-5:]):
    print(f'{p.scores[s.s_id].value*0.01:5.2f}% on {s} (Quality parameter: {v:0.3f})')
print(f'\nTop 5 Improvement Opportunities for {p.name} (#{p.e_id}):')
for s, v in p_ranking[:5]:
    print(f'{p.scores[s.s_id].value*0.01:5.2f}% on {s} (Quality parameter: {v:0.3f})')

In [ ]:
for i, p in itl.players.items():
    if 'Viva' in p.name:
        print(f'{i}: {p.name}')

In [ ]:
check_ends = 5
rel_ends = []
for s_lo in ordering_rev[1:check_ends]:
    for s_hi in ordering_rev[-check_ends:-1]:
        r = itl.relationship_lookup(s_lo, s_hi, allow_link=True)
        print(r)
        rel_ends.append(r)

In [ ]:
def quick_plot(r: Relationship):
    x_scores = [r.x.scores[e_id].value for e_id in r.e_common]
    y_scores = [r.y.scores[e_id].value for e_id in r.e_common]

    ex_matrix = 1 - np.vstack([x_scores, y_scores]) * Relationship.SCORE_SCALAR

    screen_max_neg = np.amax(ex_matrix, axis=0)
    screen_min_neg = np.amin(ex_matrix, axis=0)

    ex_matrix_screen = ex_matrix[:, np.logical_and(
        ex_matrix[0, :] < 0.10,
        ex_matrix[1, :] < 0.20,
        screen_min_neg > Relationship.MIN_NEG_LIMIT
    )]

    x_all = ex_matrix_screen[0, :]
    y_all = ex_matrix_screen[1, :]

    plt.scatter(x_all, y_all)
    plt.show()

quick_plot(rel_ends[5])